In [1]:
import pandas as pd
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
import numpy as np
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler

In [2]:
df = pd.read_parquet("../data/static_models/pump4_power_siso.par")
df = df.set_index("time")
df = df.rolling(window=40, center=True).median().dropna()
df = df.resample("1min").mean().dropna()

In [3]:
df['pump4_speed'] = df['pump4_speed'].apply(lambda x: max(0, x))


In [4]:
train_df = df["2024-01-09 13:00:00":"2024-01-09 17:00:00"]
test_df = df["2024-01-10":"2024-02-16 12:00:00"]

In [5]:
from src.system_identification import ARXParametersIdentification
from src.preprocessing import Preprocessor
import pandas as pd
from sysidentpy.basis_function._basis_function import Polynomial

In [6]:
FEATURES = "pump4_speed"
TARGET = 'pump4_power'

sys_id = ARXParametersIdentification(FEATURES, 
                            TARGET, 
                            dataframe= train_df, 
                            model_type= 'siso')

In [7]:
parameters = {
    'estimator': 'ridge_regression',
    'ridge_param': 2e4,
    'order_selection': False,
    'n_terms': 4,
    'extended_least_squares': False,
    'ylag': 2,  
    'xlag': 1, 
    'basis_function': Polynomial(degree=1)
}
    
fitted_model, model_parameters = sys_id.identify_arx_parameters(x_train=train_df[FEATURES], 
                                                                y_train= train_df[TARGET],
                                                                **parameters)
    
model_parameters

/home/alqua/Git/PumpStationMPC/sys_identification/src/sysidentpy/narmax_base.py:1034: RuntimeWarning: divide by zero encountered in divide
  x = x[1:] / aux_b
/home/alqua/Git/PumpStationMPC/sys_identification/src/sysidentpy/narmax_base.py:1034: RuntimeWarning: invalid value encountered in divide
  x = x[1:] / aux_b


,Regressors,Parameters,ERR
0,x1(k-1),4.6049E-02,9.9085E-01
1,y(k-1),3.7969E-02,NAN
2,y(k-2),-1.1342E-02,NAN
3,1,1.6483E-03,NAN


In [8]:
yhat  = sys_id.predict(x_test= test_df[FEATURES], y_test= test_df[TARGET], n_steps_ahead= 200)

#val_plots = sys_id.residuals_analysis(yhat, df[TARGET].values)
#print(model_parameters)

RRSE: 0.17318099946120735
MSE: 16.91446174056045


In [9]:
test_df["yhat"] = yhat

sys_id.plot_features_and_target(test_df, ['yhat', 'pump4_power'])

FigureWidgetResampler({
    'data': [{'name': '<b style="color:sandybrown">[R]</b> yhat <i style="color:#fc9944">~54m</i>',
              'showlegend': True,
              'type': 'scattergl',
              'uid': '76b53c8e-75bf-4db9-9338-430096df483d',
              'x': array([datetime.datetime(2024, 1, 10, 0, 0, tzinfo=<UTC>),
                          datetime.datetime(2024, 1, 10, 0, 28, tzinfo=<UTC>),
                          datetime.datetime(2024, 1, 10, 0, 30, tzinfo=<UTC>), ...,
                          datetime.datetime(2024, 2, 16, 9, 38, tzinfo=<UTC>),
                          datetime.datetime(2024, 2, 16, 10, 32, tzinfo=<UTC>),
                          datetime.datetime(2024, 2, 16, 11, 26, tzinfo=<UTC>)], dtype=object),
              'y': array([ 4.58600005e+01, -3.53767246e-01,  4.06562761e-03, ...,  1.68101504e-03,
                           1.69336219e-03,  1.69336219e-03])},
             {'name': ('<b style="color:sandybrown">[R' ... 'style="color:#fc9944">~54m<

In [10]:
lower_mpc_data = pd.read_parquet("../data/sym_data/sim_data_1s_with_eff_power2.parquet")


In [11]:
lower_mpc_data_p4_power = sys_id.predict(lower_mpc_data[FEATURES], lower_mpc_data[TARGET], n_steps_ahead= 1)

RRSE: nan
MSE: nan


In [12]:
lower_mpc_data["pump4_power_eff"] = lower_mpc_data_p4_power

In [13]:
#lower_mpc_data.to_parquet(("../data/sym_data/sim_data_1s_with_eff_power.parquet"))

In [14]:
r = pd.read_parquet("/home/alqua/Git/PumpStationMPC/sys_identification/results_temp_p1.par")

In [17]:
r

,pump1_rpm,pump4_rpm,pump1_power,pump4_power,outflow,pump1_power_hat
time,,,,,,
2022-11-29 11:00:00+00:00,0.000000,863.036058,0.000000,34.554111,270.834135,0.000000
2022-11-29 12:00:00+00:00,0.000000,858.861426,0.000000,34.251811,265.665316,0.000000
2022-11-29 13:00:00+00:00,804.826121,203.407701,30.865947,8.344101,276.260818,0.000000
2022-11-29 14:00:00+00:00,1040.080754,0.000000,38.845155,0.000000,262.544646,37.255642
2022-11-29 15:00:00+00:00,793.004365,185.068502,28.515796,7.675296,246.440228,48.239104
...,...,...,...,...,...,...
2023-03-09 12:00:00+00:00,1086.508333,0.000000,43.532661,0.000000,349.255278,7.857715
2023-03-09 13:00:00+00:00,1089.282500,0.000000,43.639767,0.000000,349.847222,50.416191
2023-03-09 14:00:00+00:00,372.660556,607.251667,14.865019,26.212881,358.872778,50.623095


In [16]:
FEATURES

'pump4_speed'

In [20]:
p4_hat = sys_id.predict(r["pump4_rpm"], r[TARGET], n_steps_ahead= 1)

RRSE: nan
MSE: nan


In [24]:
r["pump4_power_hat"] = p4_hat

In [25]:
r.to_parquet("evaluation_results.par")